# Tag Calculator
This notebook calculates the different unequal splits for each of the splitters given a code length and SAW filter insertion loss.

### Basic Tag Diagram
![Basic Tag Diagram](Tag-Diagram.png "Basic Tag Diagram")
This image shows a tag with a code length of 4.

In [6]:
import tabulate
import numpy as np

In [1]:
# Inputs
inst_code_length = 8
inst_insertion_loss = 1.2 #dB

In [18]:
def calc_splitters(code_length, insertion_loss):
    power_per_tap = (1-power_loss)/code_length #The amount of power we want out of each tap

    #For each splitter, calculate s parameters and output powers
    input_power = 1
    data = np.zeros((code_length, 5))
    headers = ["Splitter", "S21", "S31", "Power (P2)", "Power (P3)"]
    
    for filter in range(code_length):
        data[filter][3] = input_power - power_per_tap #Power going into the next filter
        data[filter][4] = power_per_tap               #Power going out of this tap to the transmit antenna
        data[filter][2] = data[filter][4]/input_power #S31
        data[filter][1] = data[filter][3]/input_power #S21
        data[filter][0] = filter                      #Filter IDX
        input_power = data[filter][3] * 10**-(insertion_loss/10) #loss from the next filter    

    table = tabulate.tabulate(data, headers, tablefmt='html')
    return table

In [19]:
calc_splitters(8, 1.2)

Splitter,S21,S31,Power (P2),Power (P3)
0,0.875,0.125,0.875,0.125
1,0.811678,0.188322,0.538755,0.125
2,0.694143,0.305857,0.283688,0.125
3,0.419143,0.580857,0.0901992,0.125
4,-0.826869,1.82687,-0.0565769,0.125
5,3.91253,-2.91253,-0.167918,0.125
6,1.98132,-0.981325,-0.252379,0.125
7,1.65292,-0.652916,-0.316449,0.125
